In [4]:
import pathlib

import numpy as np
import pandas as pd
import face_recognition
import faiss
import tqdm

### Get img files

In [3]:
def read_file(path):
    img_path = []
    for file in pathlib.Path(path).iterdir():
        if file.suffix == ".jpg":
            img_path.append(str(file))
    return img_path

In [ ]:
ROOTDATA = "./images"
train_imgs = read_file(ROOTDATA)
print("Total number of images: ", len(train_imgs))

In [ ]:
df = pd.read_csv("../query_data/query.csv")
print("Total data of query: ", len(df))

### Get face and Encode img with face_recognition

In [ ]:
faceEncode = []
train_labels = []
n_encoded = 0

for i in tqdm(range(len(df))):
    path = pathlib.Path(f"{ROOTDATA}/{i}.jpg")
    if not path.exists():
        # add dummy img
        face_encode = np.zeros((128,))
    else:
        # read img
        img = face_recognition.load_image_file(path)
        # detect face
        face_location = face_recognition.face_locations(img)
        if len(face_location) > 1:
            print(path)
            # add dummy img
            face_encode = np.zeros((128,))
        if len(face_location) > 0:
            #crop face
            face_encode = face_recognition.face_encodings(img, face_location)[0]
            n_encoded += 1

    faceEncode.append(face_encode)
    train_labels.append(i)

print("Number of images that could be encoded: ", n_encoded)

In [ ]:
# convert encoeded vector to np.ndarray
faceEncode = np.array(faceEncode,dtype=np.float32)
print("shape of faceEncode: ", faceEncode.shape)

In [ ]:
# get img labels
train_labels = np.array(train_labels)
print("labels: ", train_labels[:10])

In [ ]:
import numpy as np

np.save("./labels.npy", train_labels)
np.save("./face_encoded.npy", faceEncode) 

###  Create index with faiss

In [2]:
import numpy as np

train_labels = np.load("./labels.npy")
faceEncode = np.load("./face_encoded.npy")

In [5]:
# create index with faiss
face_index = faiss.IndexFlatL2(128)
# add vector
face_index.add(faceEncode)

#save index
faiss.write_index(face_index,'./face_index.bin')